In [1]:
import json
import requests



In [ ]:
# --- 1. Paramètres utilisateur ---
article = "Marine Le Pen"  
lang    = "fr"               


In [ ]:

# --- 2. Récupérer le rev_id de la dernière révision ---
page_api_url = f'https://{lang}.wikipedia.org/w/api.php'
params = {
    'action': 'query',
    'format': 'json',
    'titles': article,
    'prop': 'revisions',
    'rvprop': 'ids',
    'rvlimit': 1
}
resp = requests.get(page_api_url, params=params)
resp.raise_for_status()
data = resp.json()

pages = data.get('query', {}).get('pages', {})
page = next(iter(pages.values()))

if 'missing' in page:
    raise ValueError(f"L’article « {article} » n’existe pas sur {lang}.wikipedia.org")

rev_id = page['revisions'][0]['revid']
print(f"Dernier rev_id pour « {article} » : {rev_id}")


Page : {'pageid': 86757, 'ns': 0, 'title': 'Marine Le Pen', 'revisions': [{'revid': 225312137, 'parentid': 225245649}]}
Dernier rev_id pour « Marine Le Pen » : 225312137


In [4]:

# --- 3. Appel au modèle de lisibilité ---
inference_url = 'https://api.wikimedia.org/service/lw/inference/v1/models/readability:predict'
headers = {
    'Content-Type': 'application/json',
    'User-Agent': 'VOTRE_APP (VOTRE_CONTACT)',
}
payload = {"rev_id": rev_id, "lang": lang}

response = requests.post(inference_url, headers=headers, data=json.dumps(payload))
response.raise_for_status()
full = response.json()


In [5]:


# --- 4. Extraction des scores ---
output = full.get("output", {})
score = output.get("score")
fk_proxy = output.get("fk_score_proxy")


In [6]:

# --- 5. Construction et affichage du résultat final ---
result = {
    "rev_id": rev_id,
    "title": article,
    "readability_score": score,
    "fk_score_proxy": fk_proxy
}

print(json.dumps(result, indent=2))


{
  "rev_id": 225312137,
  "title": "Marine Le Pen",
  "readability_score": 5.021496295928955,
  "fk_score_proxy": 12.98340615510051
}
